# Split and Deduplicate the method Dataset (0x01)

(C) Maxim Gansert, Mindscan Engineering, 2020

In [ ]:
import os
import pandas as pd

This notebook will dedupliacte and split the methodNextLineDataset. The criteria for deduplication is the same methodbody and the same method name, while ignoring the method signature. The second criteria is the one for splitting; The method dataset is split using the token length of the method body as an criteria. (12 * 2^k) 0<=k<=9

Method being equal to each other must have the sam elength to be compared against each other, so splitting the methods by length before deduplicating them does no harm and speeds up the deduplication process.

12 was chosen instead of 16 because 16 produced a too big initial dataset to start with.

Why having short methods, the idea is to create a dataset which can be easily trained, e.g short datasets. Some Ideas are as short as 12 tokens, some more complex algorithms require 24 tokens and so on. Proof of concept will be trained on shortest dataset first. Method body -> method name this will reduce the amount of compute and then the embeddings can be reduced to train the larger datasets, refining the embeddings, and so on. Since the training of a transformer network is an computationally intensive method, i will start with short methods first, and then extend the input window size, to not waste compute time.

Later we also need to get rid of the same implementation leading to a different method name. But this is not our first priority right now. maybe e wan do this later.

In [ ]:
dataset_directory = 'D:\\Downloads\\Big-Code-full'
dataset_full_filename = os.path.join(dataset_directory, 'methodNextLineDataset.jsonl')

output_filename_template = 'dedup_methodNextLineDataset_{}_{}.jsonl'

# ranges where to split - two neighboring define a range of 
ranges=[0,12,24,48,96,192,384,768,1536,3072,6144]

In [ ]:
DATASET_COLUMN_FILENAME = 'file_name'
DATASET_COLUMN_CLASSNAME = 'class_name'

DATASET_COLUMN_METHODNAME = 'method_name'
DATASET_COLUMN_ENCODED_METHODNAME_LENGTH = 'length_encoded_method_name'
DATASET_COLUMN_ENCODED_METHODNAME = 'encoded_method_name'

DATASET_COLUMN_ENCODED_METHODBODY_LENGTH = 'length_encoded_method_body'
DATASET_COLUMN_ENCODED_METHODBODY = 'encoded_method_body'

DATASET_COLUMN_ENCODED_METHODSIGN_LENGTH = 'length_encoded_method_sign'
DATASET_COLUMN_ENCODED_METHODSIGN = 'encoded_method_sign'

DATASET_COLUMN_ENCODED_METHODBODY_ASTUPLE = 'encoded_method_body_tuple' #  TUPLE - TYPE is comparable in pandas
DATASET_COLUMN_ENCODED_METHODNAME_ASTUPLE = 'encoded_method_name_tuple'




In [ ]:
def head(filename, count=1):
    with open(dataset_full_filename) as myfile:
        head = [next(myfile) for x in range(count)]
    return head

print(head(dataset_full_filename))

* first filter by length ( length_encoded_method_body )
* then deduplicate by method_name, length_encoded_method_body

In [ ]:
def split_and_deduplicate_methods(fullfilepath, therange):
    result = None
    # each line contains one json document
    chunksize = 240000
    count = 0
    
    for df in pd.read_json(fullfilepath, lines=True, chunksize=chunksize ):
        count = count + 1
        print ('input dataframe size: {} {} processed'.format(df.shape, count*chunksize))
        
        df=df[df.length_encoded_method_body.between(therange[0],therange[1])]
        print ('methods in range: {}'.format(df.shape))
        
        # add a hashable version of the method body
        df[DATASET_COLUMN_ENCODED_METHODBODY_ASTUPLE] = df.encoded_method_body.apply(lambda x: tuple([item for sublist in x for item in sublist]))
        df[DATASET_COLUMN_ENCODED_METHODNAME_ASTUPLE] = df.encoded_method_name.apply(lambda x: tuple(x))
        
        df = df.drop(columns=[DATASET_COLUMN_FILENAME, DATASET_COLUMN_CLASSNAME,
                 DATASET_COLUMN_ENCODED_METHODSIGN_LENGTH, DATASET_COLUMN_ENCODED_METHODSIGN])

        # reduce dataset by removing duplicates
        reduced = df.drop_duplicates( subset=[DATASET_COLUMN_METHODNAME,
                                              DATASET_COLUMN_ENCODED_METHODBODY_LENGTH,
                                              DATASET_COLUMN_ENCODED_METHODBODY_ASTUPLE], keep='first', inplace=False)
        reduced.reset_index(inplace=True)
        
        if result is None:
            result = pd.DataFrame(columns=reduced.columns)

        print('deduplicated dataframe size: {}'.format(reduced.shape))

        result = result.append(reduced)
        result = result.drop_duplicates( subset=[DATASET_COLUMN_METHODNAME,
                                                 DATASET_COLUMN_ENCODED_METHODNAME_LENGTH,
                                              DATASET_COLUMN_ENCODED_METHODBODY_ASTUPLE], keep='first', inplace=False )

        print('result_dataframe size: {}\n\n'.format(result.shape))
        
    return result


In [ ]:
def extract_ranges(ranges, output_filename_template):
    for i in range(len(ranges)-1):
        start_of_range = ranges[0+i] + 1
        end_of_range = ranges[1+i]
        
        deduplicated = split_and_deduplicate_methods( dataset_full_filename, [ start_of_range, end_of_range ] )
        deduplicated = deduplicated.drop(columns=[DATASET_COLUMN_ENCODED_METHODBODY_ASTUPLE, DATASET_COLUMN_ENCODED_METHODNAME_ASTUPLE])

        outfile=os.path.join(dataset_directory, output_filename_template.format(start_of_range,end_of_range))
        deduplicated.to_json(outfile,orient='records',lines=True)
    pass


extract_ranges( ranges=ranges, output_filename_template=output_filename_template )